In [3]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

In [4]:
te = TransactionEncoder()
te_try = te.fit('student-mat - student-mat.csv').transform('student-mat - student-mat.csv')

In [8]:
df = pd.DataFrame(te_try, columns=te.columns_)
df.head()

,,-,.,a,c,d,e,m,n,s,t,u,v
0,False,False,False,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,False,False,True,False,False
2,False,False,False,False,False,False,False,False,False,False,False,True,False
3,False,False,False,False,False,True,False,False,False,False,False,False,False
4,False,False,False,False,False,False,True,False,False,False,False,False,False


In [6]:
from mlxtend.frequent_patterns import fpgrowth
fpgrowth(df,min_support=0.5)

,support,itemsets


In [7]:
from mlxtend.frequent_patterns import apriori
apriori(df,min_support=0.5, use_colnames=True)

,support,itemsets


In [9]:
import pandas as pd

def create_fptree(transactions, min_support):
    # Create an item count table
    item_count = {}
    for transaction in transactions:
        for item in transaction:
            if item not in item_count:
                item_count[item] = 0
            item_count[item] += 1

    # Filter items based on minimum support
    frequent_items = set([item for item, count in item_count.items() if count >= min_support])

    # Create an empty FP-tree
    fptree = FPTreeNode()

    # Create conditional FP-trees for each frequent item
    for item in frequent_items:
        conditional_transactions = []
        for transaction in transactions:
            if item in transaction:
                conditional_transactions.append([item] + [t for t in transaction if t != item and t in frequent_items])

        conditional_fptree = create_fptree(conditional_transactions, min_support)
        fptree.add_child(item, conditional_fptree, item_count[item])

    return fptree

class FPTreeNode:
    def _init_(self, item=None, count=0, parent=None):
        self.item = item
        self.count = count
        self.parent = parent
        self.children = {}

    def add_child(self, child_item, child_tree, child_count):
        if child_item not in self.children:
            self.children[child_item] = FPTreeNode(child_item, child_count, self)
        self.children[child_item].add_child_node(child_tree)

    def add_child_node(self, child_tree):
        for child_item, child_node in child_tree.children.items():
            if child_item not in self.children:
                self.children[child_item] = FPTreeNode(child_item, child_node.count, self)
            self.children[child_item].add_child_node(child_node)

def mine_frequent_patterns(fptree, prefix, min_support):
    frequent_patterns = []
    for item, child in fptree.children.items():
        pattern = prefix + [item]
        pattern_support = child.count
        if pattern_support >= min_support:
            frequent_patterns.append(pattern)
            frequent_patterns.extend(mine_frequent_patterns(child, pattern, min_support))

    return frequent_patterns

# Load the student dataset
data = pd.read_csv('student_dataset.csv')

# Convert the data into a list of transactions
transactions = [data['Courses'].values.tolist()[i] for i in range(len(data))]

# Set the minimum support threshold
min_support = 3

# Create the FP-tree
fptree = create_fptree(transactions, min_support)

# Mine frequent patterns
frequent_patterns = mine_frequent_patterns(fptree, [], min_support)

# Print the frequent patterns
print(f"Frequent patterns with minimum support {min_support}:")
for pattern in frequent_patterns:
    print(pattern)

FileNotFoundError: [Errno 2] No such file or directory: 'student_dataset.csv'

In [10]:
import numpy as np

def fp_growth(transactions, min_support):
    # Create the FP-tree
    fp_tree = create_fptree(transactions, min_support)

    # Create the prefix patterns
    prefix_patterns = []
    for item in fp_tree.items:
        prefix_patterns.append((item,))

    # Mine the prefix patterns
    for prefix_pattern in prefix_patterns:
        prefix_support = support(prefix_pattern, transactions)
        if prefix_support >= min_support:
            conditional_pattern_base = create_conditional_pattern_base(prefix_pattern, transactions, min_support)
            conditional_fp_tree = create_fptree(conditional_pattern_base, min_support)
            conditional_prefix_patterns = []
            for item in conditional_fp_tree.items:
                conditional_prefix_patterns.append((prefix_pattern, item))
            mine_prefix_patterns(conditional_prefix_patterns, transactions, min_support)

def create_fptree(transactions, min_support):
    # Create the itemset counts
    itemset_counts = {}
    for transaction in transactions:
        for item in transaction:
            if item not in itemset_counts:
                itemset_counts[item] = 0
            itemset_counts[item] += 1

    # Remove items that do not meet the minimum support
    for item, count in itemset_counts.items():
        if count < min_support:
            del itemset_counts[item]

    # Sort the items by frequency in descending order
    items = sorted(itemset_counts.keys(), key=itemset_counts.get, reverse=True)

    # Create the FP-tree
    fp_tree = FPTree()
    for transaction in transactions:
        fp_tree.add_transaction(transaction, items)

    # Return the FP-tree
    return fp_tree

def support(itemset, transactions):
    # Count the number of transactions that contain the itemset
    count = 0
    for transaction in transactions:
        if all(item in transaction for item in itemset):
            count += 1

    # Return the support of the itemset
    return count

def create_conditional_pattern_base(prefix_pattern, transactions, min_support):
    # Create the conditional pattern base
    conditional_pattern_base = []
    for transaction in transactions:
        conditional_transaction = []
        for item in transaction:
            if all(item in prefix_pattern for item in prefix_pattern) and item not in prefix_pattern:
                conditional_transaction.append(item)

        # Remove conditional transactions that do not meet the minimum support
        if support(conditional_transaction, transactions) >= min_support:
            conditional_pattern_base.append(conditional_transaction)

    # Return the conditional pattern base
    return conditional_pattern_base